In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression

from econml.sklearn_extensions.linear_model import StatsModelsLinearRegression
from econml.utilities import _safe_norm_ppf


def dr_sim(model_t, model_y, n_folds, Y, T, X, min_t=0.01):
    """
    Doubly-robust learner

    Parameters
    ----------
    model_t : object
        A classifier for the treatment.
    model_y : object
        A regressor for the outcome.
    n_folds : int
        The number of folds to use in cross-validation.
    Y : array-like
        The outcome variable.
    T : array-like
        The treatment variable.
    X : array-like
        The covariates.

    Returns
    -------
    theta : array-like
        The estimated treatment effect per treatment.
    sigma : array-like
        The estimated variance of the outcome residuals per treatment.
    nu : array-like
        The estimated variance of the treatment per treatment.
    cov : array-like
        The covariance matrix of the estimated parameters, of shape (n_treatments, 3, 3).
    """

    # Initialize KFold cross-validation
    kf = StratifiedKFold(n_splits=n_folds, shuffle=True)
    T_ohe = OneHotEncoder(sparse_output=False, drop='first').fit_transform(T.reshape(-1, 1))

    # Initialize arrays to hold predictions
    y_pred = np.zeros((Y.shape[0], T_ohe.shape[1]+1)) # include prediction for T=0
    t_pred = np.zeros((T_ohe.shape[0], T_ohe.shape[1]+1)) # include prediction for T=0
    alpha = np.zeros_like(T_ohe)

    # one value of theta, sigma, nu for each non-control treatment
    theta = np.zeros(T_ohe.shape[1])
    sigma = np.zeros(T_ohe.shape[1])
    nu = np.zeros(T_ohe.shape[1])

    # one theta, sigma, nu covariance matrix for each non-control treatment
    cov = np.zeros((T_ohe.shape[1], 3, 3))

    # Cross-validation loop
    for train_index, test_index in kf.split(X,T):
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        T_train = T[train_index]
        To_train, To_test = T_ohe[train_index], T_ohe[test_index]
     
        # Fit the treatment model
        t_pred[test_index] = model_t.fit(X_train, T_train).predict_proba(X_test)

        t_pred[test_index] = np.clip(t_pred[test_index], min_t, 1-min_t)  # avoid division by zero
        t_pred[test_index] = t_pred[test_index] / np.sum(t_pred[test_index], axis=1, keepdims=True)  # normalize to sum to 1
        
        # Fit the outcome model
        model_y.fit(np.hstack([X_train, To_train]), Y_train)
        T_hypo = np.zeros_like(To_test)
        y_pred[test_index,0] += model_y.predict(np.hstack([X_test, T_hypo]))
        for i in range(To_train.shape[1]):
            T_hypo = np.zeros_like(To_test)
            T_hypo[:,i] = 1
            y_pred[test_index,i+1] += model_y.predict(np.hstack([X_test, T_hypo]))

    # ATE, sigma^2, and nu^2
    for i in range(T_ohe.shape[1]):
        theta_score = y_pred[:,i+1] - y_pred[:,0] + (Y-y_pred[:,i+1]) * (T_ohe[:,i] == 1)/t_pred[:,i+1] - (Y-y_pred[:,0]) * (np.all(T_ohe==0, axis=1)/t_pred[:,0])
        sigma_score = (Y-np.choose(T, y_pred.T))**2 # exclude rows with other treatments
        alpha[:,i] = (T_ohe[:,i] == 1)/t_pred[:,i+1] - (np.all(T_ohe==0, axis=1))/t_pred[:,0]
        nu_score = 2*(1/t_pred[:,i+1]+1/t_pred[:,0])-alpha[:,i]**2
        theta[i] = np.mean(theta_score)
        sigma[i] = np.mean(sigma_score)
        nu[i] = np.mean(nu_score)
        scores = np.stack([theta_score-theta[i], sigma_score-sigma[i], nu_score-nu[i]], axis=1)
        cov[i,:,:] = (scores.T @ scores / len(scores)) / len(scores)

    return theta, sigma, nu, cov

def sensitivity_interval(theta, sigma, nu, cov, alpha, c_y, c_t, rho):
    """
    Calculate the sensitivity interval for a doubly-robust learner.
    """
    C = np.abs(rho) * np.sqrt(c_y) * np.sqrt(c_t/(1-c_t))/2
    ests = np.array([theta, sigma, nu])
    
    coefs_p = np.array([1, C*np.sqrt(nu/sigma), C*np.sqrt(sigma/nu)])
    coefs_n = np.array([1, -C*np.sqrt(nu/sigma), -C*np.sqrt(sigma/nu)])
    # One dimensional normal distribution:
    sigma_p = coefs_p @ cov @ coefs_p
    sigma_n = coefs_n @ cov @ coefs_n

    # print(f"theta bounds: {ests @ coefs_n}, {ests @ coefs_p}")
    # print(f"sigma bounds: {sigma_n}, {sigma_p}")

    lb = _safe_norm_ppf(alpha / 2, loc=ests @ coefs_n, scale=np.sqrt(sigma_n))
    ub = _safe_norm_ppf(1 - alpha / 2, loc=ests @ coefs_p, scale=np.sqrt(sigma_p))

    return (lb, ub)


def RV(theta, sigma, nu, cov, alpha):
    # The robustness value is the degree of confounding of *both* the treatment and the outcome that still produces an interval
    # that excludes zero.

    # We're looking for a value of r such that the sensitivity bounds just touch zero

    r = 0
    r_up = 1
    r_down = 0
    lb, ub = sensitivity_interval(theta, sigma, nu, cov, alpha, 0, 0, 1)
    if lb < 0 and ub > 0:
        return 0
    
    else:
        if lb > 0:
            target = 0
            mult = 1
            d = lb
        else:
            target = 1
            mult = -1
            d = ub

    while abs(d) > 1e-6:
        d = mult * sensitivity_interval(theta, sigma, nu, cov, alpha, r, r, 1)[target]
        if d > 0:
            r_down = r
        else:
            r_up = r

        r = (r_down + r_up) / 2
        
    return r

    


# Simulate data

In [ ]:
np.random.seed(3)
n = 10000
alpha = np.random.normal(size=5)
X = np.random.normal(size=(n,5), loc=[1,0.5,0,0,0])/3 # closest to center 1, then 2, then 3
centers = np.array([[1,0,0,0,0], [0,1,0,0,0], [0,0,1,0,0]]) # trinary treatment
# centers = np.array([[1,0,0,0,0], [0,1,0,0,0]]) # uncomment for binary treatment

ds = X[:,None,:]-centers[None,:,:]
ds = np.einsum("nci,nci->nc", ds, ds)

ps_r = np.exp(-ds)
ps = ps_r / np.sum(ps_r, axis=1, keepdims=True)

T = np.random.default_rng().multinomial(1, ps) @ np.arange(len(centers))

Y = np.random.normal(size=n) + 3*(T == 1)*X[:,1] - (T == 2) + 2 * X @ alpha

true_theta = np.mean(3*X[:,1]), -1

# Simple example

Run simple dml and calculate intermediate values

In [3]:
sig_level = 0.1

theta, sigma, nu, sig = dr_sim(LogisticRegression(), LinearRegression(), 2, Y, T, X)


#### Calculate a "sensitivity interval". 

Need to supply "strength of latent confounder" as argument.

In [4]:
for i in range(theta.shape[0]):
    lb, ub = sensitivity_interval(theta[i], sigma[i], nu[i], sig[i], sig_level, 0.6, 0.6, 1)
    print({'i': i, 'lb': lb, 'ub': ub})

{'i': 0, 'lb': -2.38862155309683, 'ub': 3.424960144685417}
{'i': 1, 'lb': -4.2305658461136355, 'ub': 2.24158270347243}


#### Calculate Robustness Value. 

The required strength of a latent confounder in order for the confidence interval to include 0.

In [5]:
for i in range(theta.shape[0]):
    rv = RV(theta[i], sigma[i], nu[i], sig[i], sig_level)
    print({'i': i, 'RV': rv})

{'i': 0, 'RV': 0.14217567443847656}
{'i': 1, 'RV': 0.24674725532531738}


# Ablations

In [ ]:
sig_level = 0.1
t_ind = 1

results = []
for i in [1, 5, 15, 30]:
    theta, sigma, nu, sig = dr_sim(LogisticRegression(), LinearRegression(), 2, Y, T, X[:,:i])
    result_dict = {
        'i': i,
        'alpha': sig_level,
        't_ind': t_ind,
        'sensitivity_interval': sensitivity_interval(theta[t_ind], sigma[t_ind], nu[t_ind], sig[t_ind], sig_level, 0.6, 0.6, 1),
        'RV': RV(theta[t_ind], sigma[t_ind], nu[t_ind], sig[t_ind], sig_level)
    }
    results.append(result_dict)
                   
results

[{'i': 1,
  'alpha': 0.1,
  't_ind': 1,
  'sensitivity_interval': (-5.548451847333972, 3.5719993434133452),
  'RV': 0.17672276496887207},
 {'i': 5,
  'alpha': 0.1,
  't_ind': 1,
  'sensitivity_interval': (-4.230585221174876, 2.2303879643275093),
  'RV': 0.2489337921142578},
 {'i': 15,
  'alpha': 0.1,
  't_ind': 1,
  'sensitivity_interval': (-4.2284220073443946, 2.23410292617903),
  'RV': 0.24792170524597168},
 {'i': 30,
  'alpha': 0.1,
  't_ind': 1,
  'sensitivity_interval': (-4.223113136185441, 2.230017984700478),
  'RV': 0.24834203720092773}]

In [7]:
(
    pd.DataFrame(results)
)

,i,alpha,t_ind,sensitivity_interval,RV
0,1,0.1,1,"(-5.548451847333972, 3.5719993434133452)",0.176723
1,5,0.1,1,"(-4.230585221174876, 2.2303879643275093)",0.248934
2,15,0.1,1,"(-4.2284220073443946, 2.23410292617903)",0.247922
3,30,0.1,1,"(-4.223113136185441, 2.230017984700478)",0.248342


# DoubleML

In [8]:
import doubleml as dml
import pandas as pd
import numpy as np

In [9]:
df = pd.concat([
    pd.DataFrame(Y).squeeze().to_frame('Y'),
    pd.DataFrame(T).squeeze().to_frame('T'),
    pd.DataFrame(X).add_prefix('X'),
], axis=1)

df.head()

,Y,T,X0,X1,X2,X3,X4
0,0.607903,0,0.215080,0.139086,-0.209000,-0.014606,-0.159073
1,-1.808441,2,-0.104622,0.461541,0.293773,0.569858,0.016678
2,-1.676531,1,0.198441,-0.015120,-0.515492,0.327456,-0.367023
3,-1.235249,2,-0.061682,0.098117,0.495383,0.078905,-0.341262
4,2.764248,1,0.095669,0.375082,-0.053504,-0.256279,-0.076677


In [10]:
dml_data = dml.DoubleMLData(df, 'Y', 'T')
dml_data

In [11]:
dml_obj = dml.DoubleMLPLR(dml_data,
                          ml_l=LinearRegression(),
                          ml_m=LinearRegression(),
                          n_folds=2,
                          score='partialling out',)
dml_obj.fit()
print(dml_obj)


================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: Y
Treatment variable(s): ['T']
Covariates: ['X0', 'X1', 'X2', 'X3', 'X4']
Instrument variable(s): None
No. Observations: 10000

------------------ Score & algorithm ------------------
Score function: partialling out

------------------ Machine learner   ------------------
Learner ml_l: LinearRegression()
Learner ml_m: LinearRegression()
Out-of-sample Performance:
Regression:
Learner ml_l RMSE: [[1.24933754]]
Learner ml_m RMSE: [[0.75572709]]

------------------ Resampling        ------------------
No. folds: 2
No. repeated sample splits: 1

------------------ Fit summary       ------------------
       coef   std err          t          P>|t|     2.5 %    97.5 %
T -0.397119  0.015463 -25.682281  1.844035e-145 -0.427425 -0.366812


In [12]:
dml_obj.sensitivity_analysis(cf_y=0.03, cf_d=0.03, rho=1.)
print(dml_obj.sensitivity_summary)


================== Sensitivity Analysis ==================

------------------ Scenario          ------------------
Significance Level: level=0.95
Sensitivity parameters: cf_y=0.03; cf_d=0.03, rho=1.0

------------------ Bounds with CI    ------------------
   CI lower  theta lower     theta  theta upper  CI upper
0 -0.471507       -0.446 -0.397119    -0.348237 -0.322857

------------------ Robustness Values ------------------
   H_0     RV (%)    RVa (%)
0  0.0  21.873191  20.629383


In [13]:
sens_benchmark = dml_obj.sensitivity_benchmark(benchmarking_set=["X4"])
print(sens_benchmark)


       cf_y      cf_d  rho  delta_theta
T  0.019825  0.000095 -1.0      -0.0022


# New datasets

### DoubleML confounded synthetic data

In [14]:
from doubleml.datasets import make_confounded_plr_data, make_confounded_irm_data

In [15]:
cf_y = 0.1
cf_d = 0.1
theta = 5.0
dpg_dict = make_confounded_irm_data(n_obs=10000, cf_y=cf_y, cf_d=cf_d, theta=theta)

c:\Users\fabiovera\AppData\Local\anaconda3\envs\dev_env2\lib\site-packages\doubleml\datasets.py:1050: UserWarning: Propensity score is close to 0 or 1. Trimming is at 0.01 and 0.99 is applied
  warnings.warn(f'Propensity score is close to 0 or 1. '


In [16]:
x_cols = [f'X{i + 1}' for i in np.arange(dpg_dict['x'].shape[1])]
df = pd.DataFrame(np.column_stack((dpg_dict['x'], dpg_dict['y'], dpg_dict['d'])), columns=x_cols + ['y', 'd'])
dml_data = dml.DoubleMLData(df, 'y', 'd')

In [17]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [18]:
dml_obj = dml.DoubleMLIRM(dml_data,
                          ml_g=RandomForestRegressor(),
                          ml_m=RandomForestClassifier(),
                          n_folds=2,
                          score='ATE',)
dml_obj.fit()
print(dml_obj)


================== DoubleMLIRM Object ==================

------------------ Data summary      ------------------
Outcome variable: y
Treatment variable(s): ['d']
Covariates: ['X1', 'X2', 'X3', 'X4', 'X5']
Instrument variable(s): None
No. Observations: 10000

------------------ Score & algorithm ------------------
Score function: ATE

------------------ Machine learner   ------------------
Learner ml_g: RandomForestRegressor()
Learner ml_m: RandomForestClassifier()
Out-of-sample Performance:
Regression:
Learner ml_g0 RMSE: [[1.08528722]]
Learner ml_g1 RMSE: [[1.14348731]]
Classification:
Learner ml_m Log Loss: [[0.67196054]]

------------------ Resampling        ------------------
No. folds: 2
No. repeated sample splits: 1

------------------ Fit summary       ------------------
       coef   std err          t  P>|t|   2.5 %   97.5 %
d  5.145625  0.062208  82.716619    0.0  5.0237  5.26755


c:\Users\fabiovera\AppData\Local\anaconda3\envs\dev_env2\lib\site-packages\doubleml\utils\_checks.py:205: UserWarning: Propensity predictions from learner RandomForestClassifier() for ml_m are close to zero or one (eps=1e-12).
  warnings.warn(f'Propensity predictions from learner {str(learner)} for'


In [19]:
dml_obj.sensitivity_analysis(cf_y=0.1, cf_d=0.1, rho=1.)
print(dml_obj.sensitivity_summary)


================== Sensitivity Analysis ==================

------------------ Scenario          ------------------
Significance Level: level=0.95
Sensitivity parameters: cf_y=0.1; cf_d=0.1, rho=1.0

------------------ Bounds with CI    ------------------
   CI lower  theta lower     theta  theta upper  CI upper
0  4.858139     4.961768  5.145625     5.329482  5.466168

------------------ Robustness Values ------------------
   H_0     RV (%)    RVa (%)
0  0.0  90.573845  84.951031


In [ ]:
sig_level=0.05
t_ind = 0

theta, sigma, nu, sig = dr_sim(
    RandomForestClassifier(), 
    RandomForestRegressor(), 
    2, 
    dpg_dict['y'],
    dpg_dict['d'].astype(int),
    dpg_dict['x']
)

result_dict = {
    'alpha': sig_level,
    'sensitivity_interval': sensitivity_interval(theta[t_ind], sigma[t_ind], nu[t_ind], sig[t_ind], sig_level, 0.6, 0.6, 1),
    'RV': RV(theta[t_ind], sigma[t_ind], nu[t_ind], sig[t_ind], sig_level)
}
result_dict

{'alpha': 0.05,
 'sensitivity_interval': (2.567683475272766, 7.648542718465433),
 'RV': 0.8172124624252319}

### 401k data

In [51]:
dml_data = dml.datasets.fetch_401K()

model selection for propensity model.

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid for model selection
# Define parameter grid for model selection
param_grid = {
    'C': [0.1, 1, 10, 100]  # For LogisticRegression
}

# First find the best LogisticRegression model
lr_grid_search = GridSearchCV(
    estimator=LogisticRegression(max_iter=1000),
    param_grid=param_grid,
    cv=3,
    scoring='accuracy'
)
lr_grid_search.fit(dml_data.x, dml_data.d.astype(int))
best_lr = lr_grid_search.best_estimator_
best_lr_score = lr_grid_search.best_score_

# Find the best RandomForestClassifier
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20]
}
rf_grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=rf_param_grid,
    cv=3,
    scoring='accuracy'
)
rf_grid_search.fit(dml_data.x, dml_data.d.astype(int))
best_rf = rf_grid_search.best_estimator_
best_rf_score = rf_grid_search.best_score_

# Choose the overall best model
if best_lr_score >= best_rf_score:
    grid_search = lr_grid_search
    print(f"LogisticRegression selected with score: {best_lr_score:.4f}")
else:
    grid_search = rf_grid_search
print(f"RandomForestClassifier selected with score: {best_rf_score:.4f}")

# Use the best estimator from grid search
best_classifier = grid_search.best_estimator_
print('401k data treatment model selection:')
print(f"Best classifier: {best_classifier}")
print(f"Best score: {grid_search.best_score_:.4f}")
print(f"best rf score: {best_rf_score:.4f}")
print(f"best lr score: {best_lr_score:.4f}")



RandomForestClassifier selected with score: 0.6435
401k data treatment model selection:
Best classifier: RandomForestClassifier(max_depth=10, n_estimators=200, random_state=42)
Best score: 0.6435
best rf score: 0.6435
best lr score: 0.6578


In [71]:
dml_obj = dml.DoubleMLIRM(dml_data,
                          ml_g=RandomForestRegressor(),
                          ml_m=best_classifier,
                          n_folds=5,
                          score='ATE')
dml_obj.fit()
print(dml_obj)

================== DoubleMLIRM Object ==================

------------------ Data summary      ------------------
Outcome variable: inuidur1
Treatment variable(s): ['tg']
Covariates: ['female', 'black', 'othrace', 'dep1', 'dep2', 'q2', 'q3', 'q4', 'q5', 'q6', 'agelt35', 'agegt54', 'durable', 'lusd', 'husd']
Instrument variable(s): None
No. Observations: 5099

------------------ Score & algorithm ------------------
Score function: ATE

------------------ Machine learner   ------------------
Learner ml_g: RandomForestRegressor()
Learner ml_m: RandomForestClassifier(max_depth=10, n_estimators=200, random_state=42)
Out-of-sample Performance:
Regression:
Learner ml_g0 RMSE: [[1.27853275]]
Learner ml_g1 RMSE: [[1.29332229]]
Classification:
Learner ml_m Log Loss: [[0.6649765]]

------------------ Resampling        ------------------
No. folds: 5
No. repeated sample splits: 1

------------------ Fit summary       ------------------
        coef   std err         t     P>|t|     2.5 %    97.5 %

In [72]:
dml_obj.sensitivity_analysis(cf_y=0.04, cf_d=0.04, rho=1.)
print(dml_obj.sensitivity_summary)


================== Sensitivity Analysis ==================

------------------ Scenario          ------------------
Significance Level: level=0.95
Sensitivity parameters: cf_y=0.04; cf_d=0.04, rho=1.0

------------------ Bounds with CI    ------------------
   CI lower  theta lower     theta  theta upper  CI upper
0 -0.246676    -0.174045 -0.074699     0.024648  0.096968

------------------ Robustness Values ------------------
   H_0    RV (%)  RVa (%)
0  0.0  3.022945  0.09602


In [ ]:
sig_level=0.05
t_ind = 0

theta, sigma, nu, sig = dr_sim(
    best_classifier, 
    RandomForestRegressor(), 
    5, 
    dml_data.y,
    dml_data.d.astype(int),
    dml_data.x
)

result_dict = {
    'alpha': sig_level,
    'sensitivity_interval': sensitivity_interval(theta[t_ind], sigma[t_ind], nu[t_ind], sig[t_ind], sig_level, 0.00001, 0.0001, 1),
    'RV': RV(theta[t_ind], sigma[t_ind], nu[t_ind], sig[t_ind], sig_level)
}
result_dict

{'alpha': 0.05,
 'sensitivity_interval': (-0.15765123322817087, 0.01212238369880464),
 'RV': 0}

### bonus data

In [77]:
dml_data = dml.datasets.fetch_bonus()

model selection for propensity model

In [78]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid for model selection
# Define parameter grid for model selection
param_grid = {
    'C': [0.1, 1, 10, 100]  # For LogisticRegression
}

# First find the best LogisticRegression model
lr_grid_search = GridSearchCV(
    estimator=LogisticRegression(max_iter=1000),
    param_grid=param_grid,
    cv=3,
    scoring='neg_log_loss'
)
lr_grid_search.fit(dml_data.x, dml_data.d.astype(int))
best_lr = lr_grid_search.best_estimator_
best_lr_score = lr_grid_search.best_score_

# Find the best RandomForestClassifier
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20]
}
rf_grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=rf_param_grid,
    cv=3,
    scoring='neg_log_loss'
)
rf_grid_search.fit(dml_data.x, dml_data.d.astype(int))
best_rf = rf_grid_search.best_estimator_
best_rf_score = rf_grid_search.best_score_

# Choose the overall best model
# if best_lr_score >= best_rf_score:
#     grid_search = lr_grid_search
#     print(f"LogisticRegression selected with score: {best_lr_score:.4f}")
# else:
grid_search = rf_grid_search
print(f"RandomForestClassifier selected with score: {best_rf_score:.4f}")

# Use the best estimator from grid search
best_classifier = grid_search.best_estimator_
print('bonus data treatment model selection:')
print(f"Best classifier: {best_classifier}")
print(f"Best score: {grid_search.best_score_:.4f}")
print(f"best rf score: {best_rf_score:.4f}")
print(f"best lr score: {best_lr_score:.4f}")



RandomForestClassifier selected with score: -0.6624
bonus data treatment model selection:
Best classifier: RandomForestClassifier(max_depth=10, n_estimators=200, random_state=42)
Best score: -0.6624
best rf score: -0.6624
best lr score: -0.6442


In [79]:
dml_obj = dml.DoubleMLIRM(dml_data,
                          ml_g=RandomForestRegressor(),
                          ml_m=best_classifier,
                          n_folds=2,
                          score='ATE',)
dml_obj.fit()
print(dml_obj)


================== DoubleMLIRM Object ==================

------------------ Data summary      ------------------
Outcome variable: inuidur1
Treatment variable(s): ['tg']
Covariates: ['female', 'black', 'othrace', 'dep1', 'dep2', 'q2', 'q3', 'q4', 'q5', 'q6', 'agelt35', 'agegt54', 'durable', 'lusd', 'husd']
Instrument variable(s): None
No. Observations: 5099

------------------ Score & algorithm ------------------
Score function: ATE

------------------ Machine learner   ------------------
Learner ml_g: RandomForestRegressor()
Learner ml_m: RandomForestClassifier(max_depth=10, n_estimators=200, random_state=42)
Out-of-sample Performance:
Regression:
Learner ml_g0 RMSE: [[1.28597712]]
Learner ml_g1 RMSE: [[1.31606153]]
Classification:
Learner ml_m Log Loss: [[0.66950577]]

------------------ Resampling        ------------------
No. folds: 2
No. repeated sample splits: 1

------------------ Fit summary       ------------------
        coef   std err        t     P>|t|     2.5 %    97.5 %

In [80]:
dml_obj.sensitivity_analysis(cf_y=0.04, cf_d=0.04, rho=1.)
print(dml_obj.sensitivity_summary)


================== Sensitivity Analysis ==================

------------------ Scenario          ------------------
Significance Level: level=0.95
Sensitivity parameters: cf_y=0.04; cf_d=0.04, rho=1.0

------------------ Bounds with CI    ------------------
   CI lower  theta lower     theta  theta upper  CI upper
0 -0.232833    -0.157464 -0.064992     0.027481  0.103315

------------------ Robustness Values ------------------
   H_0    RV (%)   RVa (%)
0  0.0  2.828461  0.000603


In [ ]:
sig_level=0.05
t_ind = 0

theta, sigma, nu, sig = dr_sim(
    best_classifier, 
    RandomForestRegressor(), 
    5, 
    dml_data.y,
    dml_data.d.astype(int),
    dml_data.x
)

result_dict = {
    'alpha': sig_level,
    'sensitivity_interval': sensitivity_interval(theta[t_ind], sigma[t_ind], nu[t_ind], sig[t_ind], sig_level, 0.05, 0.05, 1),
    'RV': RV(theta[t_ind], sigma[t_ind], nu[t_ind], sig[t_ind], sig_level)
}
result_dict

{'alpha': 0.05,
 'sensitivity_interval': (-0.29151616548820364, 0.1365988298703495),
 'RV': 0}